In [30]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [179]:
# espn site text
t1 = requests.get('http://games.espn.com/fba/playerrater?leagueId=214887&teamId=9')
t2 = requests.get("http://games.espn.com/fba/playerrater?leagueId=214887&teamId=9&startIndex=50")
t3 = requests.get("http://games.espn.com/fba/playerrater?leagueId=214887&teamId=9&startIndex=100")
t4 = requests.get("http://games.espn.com/fba/playerrater?leagueId=214887&teamId=9&startIndex=150")
t5 = requests.get("http://games.espn.com/fba/playerrater?leagueId=214887&teamId=9&startIndex=200")
t6 = requests.get("http://games.espn.com/fba/playerrater?leagueId=214887&teamId=9&startIndex=250")
t7 = requests.get("http://games.espn.com/fba/playerrater?leagueId=214887&teamId=9&startIndex=300")
t8 = requests.get("http://games.espn.com/fba/playerrater?leagueId=214887&teamId=9&startIndex=350")
t9 = requests.get("http://games.espn.com/fba/playerrater?leagueId=214887&teamId=9&startIndex=400")

In [180]:
ranks = [t1,t2,t3,t4,t5,t6,t7,t8,t9]
pt=[]
for r in ranks:
    pt.append(r.text)

In [220]:
rank_tbls=[]
for text in pt:
    bs=BeautifulSoup(text,"lxml")
    table = bs.find(lambda tag: tag.name=='table' and tag.has_key('id') and tag['id']=="playertable_0")
    df = pd.read_html(str(table))
    ranktable = df[0].iloc[1:len(df[0]),[0,1,3,6,7,8,9,10,11,12,13,15]]
    colnames = ranktable.iloc[0,0:(len(ranktable.columns)-1)]
    ranktable.columns = list(colnames)+["2018 PlayerRating"]
    rankings = ranktable.iloc[1:len(ranktable),:].set_index("RNK")
    rankings["position"] = rankings["PLAYER, TEAM POS"].apply(lambda p: p[-2:])
    rankings["PLAYER, TEAM POS"] = rankings["PLAYER, TEAM POS"].apply(lambda p: p.split(",")[0])
    rank_tbls.append(rankings)
    

In [221]:
rankings = pd.concat(rank_tbls)

In [222]:
league = ['WONG', 'Firo', 'PHAN', 'JETH', 'DHIN', 'MONE', 'HDP', 'cc', 'SYED', 'BOBK']

In [223]:
team_players={}
buckets=[]
for owner in league:
    team = rankings.loc[rankings["TYPE"]==owner]
    teamtop50 = len(rank_tbls[0].loc[rankings["TYPE"]==owner])
    teamtop100 = len(rank_tbls[1].loc[rankings["TYPE"]==owner])+teamtop50
    teamtop150 = len(rank_tbls[2].loc[rankings["TYPE"]==owner])+teamtop100
    teamtop200 = len(rank_tbls[3].loc[rankings["TYPE"]==owner])+teamtop150
    
    team_players[owner] = [sum(team["2018 PlayerRating"]), teamtop50,
                           teamtop100,teamtop150,teamtop200]

In [224]:
teamperformance = pd.DataFrame(pd.Series(team_players))
teamperformance["TeamRating"] = teamperformance[0].apply(lambda s: s[0])
teamperformance["nTop50Players"] = teamperformance[0].apply(lambda s: s[1])
teamperformance["nTop100Players"] = teamperformance[0].apply(lambda s: s[2])
teamperformance["nTop150Players"] = teamperformance[0].apply(lambda s: s[3])
teamperformance["nTop200Players"] = teamperformance[0].apply(lambda s: s[4])
standings = teamperformance.iloc[:,1:len(teamperformance.columns)].sort("TeamRating",ascending=False)

/Users/eugenewong/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:7: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [225]:
standings

,TeamRating,nTop50Players,nTop100Players,nTop150Players,nTop200Players
Firo,93.85,8,12,12,12
WONG,91.66,7,10,13,13
HDP,88.32,5,12,13,13
SYED,71.04,6,9,10,10
PHAN,69.43,4,5,10,13
MONE,64.34,6,9,9,9
JETH,57.70,3,7,9,12
BOBK,51.97,3,8,12,12
DHIN,47.42,3,4,8,10
cc,44.91,5,8,9,9


In [227]:
rankings.loc[rankings["TYPE"]=="WONG"]

,"PLAYER, TEAM POS",TYPE,FG%,FT%,3PM,REB,AST,STL,BLK,PTS,2018 PlayerRating,position
RNK,,,,,,,,,,,,
1,LeBron James,WONG,4.80,1.36,0.80,1.50,4.24,1.49,2.21,3.25,19.65,PF
17,Al Horford,WONG,1.62,0.58,1.28,2.56,2.05,0.27,1.06,1.32,10.74,C
30,Jayson Tatum,WONG,0.90,0.85,1.28,1.46,0.30,0.47,1.52,1.26,8.05,SF
31,Joel Embiid,WONG,1.49,-0.08,-0.15,1.83,0.75,1.09,1.75,1.36,8.04,TD
33,Eric Gordon,WONG,-0.92,1.63,3.79,-0.18,0.98,0.07,-0.10,2.38,7.64,SG
39,Steven Adams,WONG,3.24,-0.43,-0.87,1.61,-0.15,1.29,1.98,0.57,7.25,C
49,Tyreke Evans,WONG,0.82,1.26,1.64,0.73,0.58,0.47,-0.79,1.47,6.19,SG
52,Rondae Hollis-Jefferson,WONG,1.12,1.56,-0.63,0.66,-0.15,0.88,1.52,1.07,6.04,TD
55,DeMar DeRozan,WONG,0.27,1.34,-0.27,0.07,1.03,1.70,-0.56,2.14,5.72,SG
